In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end


using MorphoMol
using JLD2
using LinearAlgebra
using Rotations
using Printf

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [54]:
function get_aggregate_simulation_evaluation(folder)
    mol_type = "6r7m"
    assembly_successes = 0
    total_simulations = 0
    id_index = 1
    steps_from_init_to_success = Vector{Int}([])
    steps_from_last_max_to_success = Vector{Int}([])

    for file in readdir(folder)
        if split(file, ".")[end] == "jld2"
            try
                @load "$folder$file" input output

                E_min_index = argmin(output["Es"])
                E_max_index = argmax(output["Es"])

                # Crude test to exclude runs that remained dispersed
                if maximum(output["Es"]) - minimum(output["Es"]) < 10.0
                    println("E_diff = $(output["Es"][E_max_index] - output["Es"][E_min_index])")
                    continue
                end

                exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
                exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
                theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
                

                E_max = maximum(output["Es"])
                last_E_disp_index = E_max_index > E_min_index ? 1 : findall(x->x≈E_max, output["Es"][1:E_min_index])[end]
    
                total_simulations += 1
                if theta < 2.0
                    assembly_successes += 1
                    push!(steps_from_init_to_success, E_min_index)
                    push!(steps_from_last_max_to_success, E_min_index - last_E_disp_index)
                end
            catch e
                println("Error in $file")
            end
        end
    end
    return total_simulations, assembly_successes/total_simulations, sum(steps_from_init_to_success)/length(steps_from_init_to_success), sum(steps_from_last_max_to_success)/length(steps_from_init_to_success)
end

get_aggregate_simulation_evaluation (generic function with 1 method)

In [55]:
folder = "../../Data/collected_simulation_results/rwm_ma_2_6r7m/"
ts, sr, sfits, sfmts = get_aggregate_simulation_evaluation(folder)
println("Morphometric Approach")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps from init to success: %.0f, Steps from last max to success: %.0f\n", ts, sr, sfits, sfmts)

Morphometric Approach
Number of simulations: 198, Success ratio: 0.056, Steps from init to success: 5002, Steps from last max to success 702


In [57]:
folder = "../../Data/collected_simulation_results/rwm_wp_2_6r7m/"
ts, sr, sfits, sfmts = get_aggregate_simulation_evaluation(folder)
println("Morphometric Approach with Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps from init to success: %.0f, Steps from last max to success: %.0f\n", ts, sr, sfits, sfmts)

Morphometric Approach with Persistence
Number of simulations: 272, Success ratio: 0.107, Steps from init to success: 561, Steps from last max to success: 547


In [59]:
folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"
ts, sr, sfits, sfmts = get_aggregate_simulation_evaluation(folder)
println("Only Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps from init to success: %.0f, Steps from last max to success: %.0f\n", ts, sr, sfits, sfmts)

Only Persistence
Number of simulations: 76, Success ratio: 0.039, Steps from init to success: 1018, Steps from last max to success: 980
